# Filtracja bilateralna

## Konwolucja obrazu z filtrem o zadanych współczynnikach

Splot (konwolucję) obrazu wejściowego $I$ z filtrem $\psi$ dla ustalonego punktu obrazu $\mathbf{x}$ można przedstawić następująco:

\begin{equation}
\hat{I}(\mathbf{x}) = \frac{1}{W_N}\sum_{\mathbf{p} \in \eta(\mathbf{x})} \psi(||\mathbf{p}-\mathbf{x}||)I(\mathbf{p})
\tag{1}
\end{equation}

gdzie:
- $\hat{I}$ jest obrazem wynikowym (przefiltrowanym),
- $W_N = \sum_y \psi(y)$ jest parametrem normalizującym współczynniki filtra $\psi$,
- $||\cdot||$ jest odległością między punktami obrazu $\mathbf{x}$ i $\mathbf{p}$ według ustalonej metryki (np. norma $L_2$). Uwaga, proszę pamiętać, że zarówno $\mathbf{x}$, jak i $\mathbf{p}$ to współrzędne przestrzenne,
- $\eta(\mathbf{x})$ jest otoczeniem punktu $\mathbf{x}$.

Funkcja $\psi$ decyduje o charakterze filtracji. Dla filtru Gaussowskiego:

\begin{equation}
\psi(y) = G_{\delta_s}(y)
\tag{2}
\end{equation}

gdzie: $G_{\delta_s}(y)$ jest funkcją Gaussa z parametrem skali $\delta_s$.

Opisaną powyżej filtrację realizowaliśmy w ramach ćwiczenia "Przetwarzanie wstępne. Filtracja kontekstowa."

## Filtracja bilateralna

Wadą klasycznego splotu jest brak adaptacji współczynników filtra do lokalnego otoczenia $\eta(\mathbf{x})$ filtrowanego punktu $\mathbf{x}$.
Oznacza to wykorzystanie tych samych współczynników filtra $\psi$ niezależnie od tego czy otoczenie jest względnie jednorodne lub zawiera krawędzie obiektów (w tym przypadku dochodzi do "rozmywania" krawędzi).
Filtracja bilateralna uwzględnia lokalne otoczenie filtrowanego punktu, w ten sposób, że parametry filtra zmieniają się w zależności od "wyglądu" otoczenia.


Współczynniki filtra obliczane są na podstawie odległości filtrowanego punktu $\mathbf{x}$ od każdego punktu otoczenia $\mathbf{p}$ w dziedzinie przestrzennej obrazu (tak jak przy typowym filtrze np. Gaussa) oraz odległości punktów w przeciwdziedzinie obrazu (np. na podstawie różnicy w jasności pikseli dla obrazu w odcieniach szarości):

\begin{equation}
\hat{I}(\mathbf{x}) = \frac{1}{W_N}\sum_{\mathbf{p} \in \eta(\mathbf{x})} \psi(||\mathbf{p}-\mathbf{x}||) \gamma(|I(\mathbf{p}) - I(\mathbf{x})|) I(\mathbf{p})
\tag{3}
\end{equation}
gdzie:
- $W_N$ jest współczynnikiem normalizującym filtr,
- $\gamma$ jest funkcją odległości w przeciwdziedzinie obrazu, np. $\gamma(y)=\exp(-\frac{y^2}{2\delta_r^2})$
- parametr $\delta_r$ jest utożsamiany z poziomem szumu w obrazie i należy go dobrać w sposób empiryczny.

Proszę chwilę zastanowić się nad powyższym równaniem, w szczególności nad funkcją $\gamma$. Proszę wyznaczyć, jaka będzie wartość funkcji dla pikseli podobnych (różnica 0, 1, 2), a skrajnie różnych (255, 200).

##  Realizacja ćwiczenia

### Wczytanie danych

1. Wczytaj dane z pliku *MR_data.mat*. W tym celu wykorzystaj funkcję `loadmat` z pakietu scipy:
        from scipy.io import loadmat
        mat = loadmat('MR_data.mat')

2. Wczytany plik zawiera 5 obrazów: *I_noisefree*, *I_noisy1*, *I_noisy2*, *I_noisy3* oraz *I_noisy4*. Odczytać je można w następujący sposób:
        Input = mat['I_noisy1']

3. Wyświetl wybrany obraz z pliku *MR_data.mat*. Zagadka - co to za obrazowanie medyczne?

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
from scipy.io import loadmat
import math
import os
import cv2
import copy

if not os.path.exists("MR_data.mat") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/07_Bilateral/MR_data.mat --no-check-certificate

mat = loadmat('MR_data.mat')
input1 = mat['I_noisy1']
input2 = mat['I_noisy2']
input3 = mat['I_noisy3']
input4 = mat['I_noisy4']
input_noisefree = mat['I_noisefree']

plt.figure(figsize=(14, 8))
ax = plt.subplot(2, 3, 1)
ax.imshow(input1, 'gray')
plt.xticks([]), plt.yticks([])
ax = plt.subplot(2, 3, 2)
ax.imshow(input2, 'gray')
plt.xticks([]), plt.yticks([])
ax = plt.subplot(2, 3, 3)
ax.imshow(input3, 'gray')
plt.xticks([]), plt.yticks([])
ax = plt.subplot(2, 3, 4)
ax.imshow(input4, 'gray')
plt.xticks([]), plt.yticks([])
ax = plt.subplot(2, 3, 5)
ax.imshow(input_noisefree, 'gray')
plt.xticks([]), plt.yticks([])
plt.show()

### "Klasyczna" konwolucja

1. Zdefiniuj parametry filtra Gaussowskiego: rozmiar okna i wariancję $\delta_S$.
2. Oblicz współczynniki filtra na podstawie zdefiniowanych parametrów (najprościej w ramach podwójnej pętli for).
2. Sprawdź ich poprawność i zwizualizuj filtr (tak jak w ćwiczeniu pt. "Przetwarzanie wstępne. Filtracja kontekstowa.").
3. Wykonaj kopię obrazu wejściowego: `IConv = Input.copy()`
4. Wykonaj podwójną pętlę po obrazie. Pomiń ramkę, dla której nie jest zdefiniowany kontekst o wybranej wielkości.
5. W każdej iteracji stwórz dwuwymiarową tablicę zawierającą aktualny kontekst.
6. Napisz funkcję, która będzie obliczała nową wartość piksela.
Argumentem tej funkcji są aktualnie przetwarzane okno i współczynniki filtra.
7. Obliczoną wartość przypisz do odpowiedniego piksela kopii obrazu wejściowego.
8. Wyświetl wynik filtracji.
9. Porównaj wynik z obrazem oryginalnym.

In [ ]:
def get_gausss_filter(window_size, delta_s):
    filter = np.zeros(shape=(window_size, window_size))
    for i in range(window_size):
        for j in range(window_size):
            y = np.sqrt(np.power(window_size//2 - i, 2) + np.power(window_size//2 - j, 2))
            gauss = np.exp(-(y**2)/(2*(delta_s**2)))
            filter[i, j] = gauss
    return filter

def calculate_pixel(window, gauss_filter):
    X, Y = window.shape
    pixel = 0
    normalization = 0
    for i in range(X):
        for j in range(Y):
            pixel += gauss_filter[i, j]*window[i, j]
            normalization += gauss_filter[i, j]
    return pixel/normalization

def classic_convolution(image, window_size, delta_s):
    copied_image = copy.copy(image)
    gauss_filter = get_gausss_filter(window_size, delta_s)
    X, Y = copied_image.shape
    half_of_window_size = window_size//2
    for i in range(0+half_of_window_size, X - half_of_window_size):
        for j in range(0 + half_of_window_size, Y - half_of_window_size):
            window = copied_image[i - half_of_window_size : i + half_of_window_size + 1, j - half_of_window_size : j + half_of_window_size + 1,]
            pixel = calculate_pixel(window, gauss_filter)
            copied_image[i,j] = pixel
    return copied_image    

In [ ]:
def plot_results(my_result, cv2_result, original_image):
    plt.figure(figsize=(10, 6))
    ax = plt.subplot(1, 3, 1)
    ax.imshow(my_result, 'gray')
    plt.xticks([]), plt.yticks([])
    plt.title('Moja funkcja')
    ax = plt.subplot(1, 3, 2)
    ax.imshow(cv2_result, 'gray')
    plt.title('Funkcja z biblioteki cv2')
    plt.xticks([]), plt.yticks([])
    ax = plt.subplot(1, 3, 3)
    ax.imshow(original_image, 'gray')
    plt.title('Oryginał')
    plt.xticks([]), plt.yticks([])
    plt.show()

def mesh(fun, size):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    X = np.arange(-size//2, size//2, 1)
    Y = np.arange(-size//2, size//2, 1)
    X, Y = np.meshgrid(X, Y)
    Z = fun
    ax.plot_surface(X, Y, Z)
    plt.show()

mesh(get_gausss_filter(3, 0.2), 3)
mesh(get_gausss_filter(5, 0.7), 5)
mesh(get_gausss_filter(7, 0.1), 7)

window_size = 5
delta_s = 0.7
input1_gauss = cv2.GaussianBlur(input1, (3, 3), delta_s)
for input_image in [input1, input2, input3, input4, input_noisefree]:
    plot_results(classic_convolution(input_image, window_size, delta_s), 
                 cv2.GaussianBlur(input_image, (window_size, window_size), delta_s), input_image)

### Filtracja bilateralna

1. Zdefiniuj dodatkowy parametr: wariancję $\delta_R$.
3. Wykonaj kopię obrazu wejściowego: `IBilateral = Input.copy()`
4. Wykonaj podwójną pętlę po obrazie. Pomiń ramkę, dla której nie jest zdefiniowany kontekst o wybranej wielkości.
5. W każdej iteracji stwórz dwuwymiarową tablicę zawierającą aktualny kontekst.
6. Napisz funkcję, która będzie obliczała nową wartość piksela.
Argumentami funkcji są aktualnie przetwarzane okno, współczynniki filtra gaussowskiego (takie same jak wcześniej) i wariancja $\delta_R$.
7. Oblicz odległość w przeciwdziedzinie (dla wartości pikseli).
8. Oblicz funkcję Gaussa dla obliczonych odległości z zadanym parametrem.
9. Wykonaj normalizację obliczonych współczynników.
10. Obliczoną wartość przypisz do odpowiedniego piksela kopii obrazu wejściowego.
11. Wyświetl wynik filtracji.
12. Porównaj wynik z obrazem oryginalnym.

In [ ]:
def calculate_pixel_bilateral(window, gauss_filter, delta_r):
    X, Y = window.shape
    pixel = 0
    normalization = 0
    for i in range(X):
        for j in range(Y):
            difference = np.abs(window[X//2, Y//2] - window[i, j])
            gauss_gamma = np.exp(-(difference**2)/(2*(delta_r**2)))
            pixel += gauss_filter[i, j]*window[i, j]*gauss_gamma
            normalization += gauss_filter[i, j]*gauss_gamma
    return pixel/normalization

def bilateral_convolution(image, window_size, delta_s, delta_r):
    copied_image = copy.copy(image)
    gauss_filter = get_gausss_filter(window_size, delta_s)
    X, Y = copied_image.shape
    half_of_window_size = window_size//2
    for i in range(0+half_of_window_size, X - half_of_window_size):
        for j in range(0 + half_of_window_size, Y - half_of_window_size):
            window = copied_image[i - half_of_window_size : i + half_of_window_size + 1, j - half_of_window_size : j + half_of_window_size + 1,]
            pixel = calculate_pixel_bilateral(window, gauss_filter, delta_r)
            copied_image[i,j] = pixel
    return copied_image 

In [ ]:
def plot_results(my_result, cv2_result, bilateral_image, original_image):
    plt.figure(figsize=(12, 12))
    ax = plt.subplot(2, 2, 1)
    ax.imshow(my_result, 'gray')
    plt.xticks([]), plt.yticks([])
    plt.title('Moja funkcja')
    ax = plt.subplot(2, 2, 2)
    ax.imshow(cv2_result, 'gray')
    plt.title('Funkcja z biblioteki cv2')
    plt.xticks([]), plt.yticks([])
    ax = plt.subplot(2, 2, 3)
    ax.imshow(bilateral_image, 'gray')
    plt.title('Filtracja bilateralna')
    plt.xticks([]), plt.yticks([])
    ax = plt.subplot(2, 2, 4)
    ax.imshow(original_image, 'gray')
    plt.title('Oryginał')
    plt.xticks([]), plt.yticks([])
    plt.show()

In [ ]:
window_size = 5
delta_s = 0.7
delta_r = 20
plot_results(classic_convolution(input1, window_size, delta_s), 
            cv2.GaussianBlur(input1, (window_size, window_size), delta_s), 
            bilateral_convolution(input1, window_size, delta_s, delta_r), input1)

In [ ]:
window_size = 7
delta_s = 0.9
delta_r = 50
plot_results(classic_convolution(input2, window_size, delta_s), 
            cv2.GaussianBlur(input2, (window_size, window_size), delta_s), 
            bilateral_convolution(input2, window_size, delta_s, delta_r), input2)

In [ ]:
window_size = 5
delta_s = 0.9
delta_r = 10
plot_results(classic_convolution(input_noisefree, window_size, delta_s), 
            cv2.GaussianBlur(input_noisefree, (window_size, window_size), delta_s), 
            bilateral_convolution(input_noisefree, window_size, delta_s, delta_r), input_noisefree)